In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
from datetime import datetime

In [2]:
df = pd.read_pickle('Data/football.pkl')

In [3]:
df['Précipitations en mm'] = df['Précipitations en mm'].astype('float32')

In [4]:
df['Date'] = df['Année'].astype(str)+'-'+df['Mois'].astype(str)+'-'+df['Jour'].astype(str)

df['Date'] = df['Date'].astype('datetime64[ns]')
df['Date'] = df['Date'].apply(lambda x : x.strftime('%Y-%m-%d'))

In [5]:
df = df.drop(columns=['Année','Mois','Jour','Heure'])

In [6]:
df.reindex(columns=['Date','Equipe domicile', 'Equipe exterieure', 'Vainqueur',
       'Score domicile', 'Score extérieur', 'Buteurs', 'Lieu du match',
       'Journée', 'Région', 'Temperature moyenne', 'Précipitations en mm'])

,Date,Equipe domicile,Equipe exterieure,Vainqueur,Score domicile,Score extérieur,Buteurs,Lieu du match,Journée,Région,Temperature moyenne,Précipitations en mm
0,2020-08-21,Bordeaux,Nantes,ex-aequo,0,0,[None],Bordeaux,1,aquitaine,22.5,5.0
1,2020-08-22,Dijon,Angers,exterieur,0,1,[I. Traoré],Dijon,1,bourgogne,22.0,19.0
2,2020-08-22,Lille,Rennes,ex-aequo,1,1,"[J. Bamba, D. Da Silva]",Lille,1,nord-pas-de-calais,19.0,1.0
3,2020-08-23,Monaco,Reims,ex-aequo,2,2,"[A. Disasi, B. Badiashile, B. Dia, E. Touré]",Monaco,1,provence-alpes-c-te-d-azur,25.5,1.0
4,2020-08-23,Lorient,Strasbourg,domicile,3,1,"[Y. Wissa, A. Grbic, P. Hamel, M. Chahiri]",Lorient,1,bretagne,17.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
375,2021-05-23,Rennes,Nîmes,domicile,2,0,"[S. Guirassy, B. Bourigeaud]",Rennes,38,bretagne,10.0,2.0
376,2021-05-23,Nantes,Montpellier,exterieur,1,2,"[R. Kolo Muani, G. Laborde, A. Delort]",Nantes,38,pays-de-la-loire,12.0,1.0
377,2021-05-23,Saint-Étienne,Dijon,exterieur,0,1,[A. Kamara],Saint-Étienne,38,rh-ne-alpes,12.0,0.0
378,2021-05-23,Reims,Bordeaux,exterieur,1,2,"[E. Touré, Y. Adli, E. Kwateng]",Reims,38,champagne-ardenne,10.5,1.0


In [7]:
con=sqlite3.connect('Data/final_db.sqlite')
cursor = con.cursor()

In [8]:
championship_list = ['Ligue1','France',2020, 2021]

In [9]:
cursor.execute("""
        INSERT INTO championships (name, country, start_year, end_year)
        VALUES (?, ?, ?, ?)""",
        championship_list)

con.commit()

In [10]:
df_teams = pd.read_csv('Data/teams.csv')
df_teams = df_teams.rename(columns={'Unnamed: 0' : 'Rank'})
df_teams['Rank'] = df_teams['Rank'].apply(lambda x: x+1)
df_teams

,Rank,Entraineur,Club
0,1,C. Galtier,Lille
1,2,M. Pochettino,Paris-SG
2,3,N. Kovac,Monaco
3,4,P. Bosz,Lyon
4,5,J. Sampaoli,Marseille
5,6,B. Genesio,Rennes
6,7,F. Haise,Lens
7,8,O. Dal Oglio,Montpellier
8,9,A. Ursea,Nice
9,10,F. Antonetti,Metz


In [11]:
teams_sql = []
for i in zip(df_teams['Club'], df_teams['Club'], df_teams['Entraineur'], df_teams['Rank']):
    teams_sql.append(i)

In [12]:
for team in teams_sql:
    con.execute("""INSERT INTO
            teams (name, city, coach_name, rank)
            VALUES (?, ?, ?, ?)""",
        team)

con.commit()

In [13]:
df_players = pd.read_csv('Data/teams_players.csv')
df_players = df_players.drop('Unnamed: 0', axis=1)
df_players = df_players.rename(columns={'Né\xa0le' : 'Birthday'})
df_players

,Numéro de mallot,Nom,Pays,Birthday,Pos.,Club,Entraineur
0,30.0,L. Chevalier,FRA,6/11/01,Gar.,Lille,C. Galtier
1,1.0,O. Karnezis,GRE,11/07/85,Gar.,Lille,C. Galtier
2,16.0,M. Maignan,FRA,3/07/95,Gar.,Lille,C. Galtier
3,5.0,S. Botman,HOL,12/01/00,Déf.,Lille,C. Galtier
4,None,I. Bouleghcha,FRA,9/12/02,Déf.,Lille,C. Galtier
...,...,...,...,...,...,...,...
619,None,M. Baldé,GUB,6/11/95,Att.,Dijon,D. Linares
620,None,E. Belhadji,FRA,23/06/01,Att.,Dijon,D. Linares
621,None,M. Chouiar,FRA,23/01/99,Att.,Dijon,D. Linares
622,None,M. Konaté,SEN,3/04/93,Att.,Dijon,D. Linares


In [14]:
# df_contracts = pd.concat([df_teams, df_players], axis=1)
# df_contracts
df_teams

,Rank,Entraineur,Club
0,1,C. Galtier,Lille
1,2,M. Pochettino,Paris-SG
2,3,N. Kovac,Monaco
3,4,P. Bosz,Lyon
4,5,J. Sampaoli,Marseille
5,6,B. Genesio,Rennes
6,7,F. Haise,Lens
7,8,O. Dal Oglio,Montpellier
8,9,A. Ursea,Nice
9,10,F. Antonetti,Metz


In [15]:
array_players = []
for i in zip(df_players['Nom'], df_players['Pos.'], df_players['Birthday'], df_players['Pays']):
    array_players.append(i)

In [16]:
for player in array_players:
    con.execute("""INSERT INTO
            players (name, position, birthdate, nationality)
            VALUES (?, ?, ?, ?)""",
        player)

con.commit()

In [18]:

matches = []
for i in zip(df['Date'], df['Lieu du match'], df['Précipitations en mm'], df['Temperature moyenne'] ):
    matches.append(i)

In [19]:
for match in matches:
    con.execute("""INSERT INTO
            matchs (date, place, rainfall, temperature, championship_id )
            VALUES (?, ?, ?, ?, (SELECT id FROM championships WHERE id = 1))""",
        match)

con.commit()

In [20]:
cursor.execute("""INSERT INTO participations (team_id, championship_id) 
SELECT teams.id, championships.id 
FROM teams 
CROSS JOIN championships;""")
con.commit()

In [21]:
list_teams = list(df_teams['Rank'])
list_clubs = list(df_teams['Club'])

In [22]:
df_contracts = df_players.replace(list_clubs, list_teams)
df_contracts = df_contracts.reset_index()

In [23]:
list_teams

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [24]:
rank_list = list(df_teams['Rank'])
rank_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [25]:
list_contracts = []
for i in zip(df_contracts['index'], df_contracts['Club']):
    list_contracts.append(i)

In [26]:
for contract in list_contracts:
    cursor.execute("""INSERT INTO
                contracts (player_id, team_id)
                VALUES (?, ?)""", 
                (contract))
con.commit()

In [27]:
cursor.execute("""SELECT COUNT(*) FROM contracts""").fetchall()

[(624,)]

In [28]:
# cursor.execute("""SELECT * FROM players""").fetchall()

In [29]:
goals_df = pd.DataFrame(df, columns=['Date','Buteurs'])
goals_df = goals_df.reset_index()
goals_df['match_id'] = goals_df['index'].apply(lambda x :x+1)

In [30]:
df_test_finl = goals_df['Buteurs'].explode()

In [31]:
indiv_buteurs = pd.DataFrame(df_test_finl)
#goals_df['final_id'] = df_test_finl.index
indiv_buteurs = indiv_buteurs.reset_index()
indiv_buteurs['index']
indiv_buteurs['index'] = indiv_buteurs['index'].apply(lambda x : x+1)
# for i in indiv_buteurs['Buteurs']:
#     if i == None:
#         print(i)
#         i = 'Test'
#         indiv_buteurs['Buteurs'] = indiv_buteurs['Buteurs']
indiv_buteurs = indiv_buteurs.fillna(value='NONE')

In [32]:
goals_sql = []
for i in zip(indiv_buteurs['Buteurs'], indiv_buteurs['index']):
    goals_sql.append(i)

In [33]:
for player in goals_sql:
    try:
        cursor.execute(f"""SELECT id FROM players WHERE name = '{player[0]}'""")
        player_id = cursor.fetchone()[0]
        print(player_id)
    except:
        player_id = -1
    cursor.execute("""INSERT INTO
                goals (player_id, match_id)
                VALUES (?, ?)""",
                (player_id, player[1]))
con.commit()

386
15
155
74
72
432
437
502
499
500
457
570
525
591
267
267
197
228
228
118
48
327
334
118
602
118
118
225
165
233
266
266
15
539
549
582
327
327
358
72
519
528
150
128
128
204
233
212
212
246
334
324
24
578
582
175
175
154
164
499
502
197
189
204
401
513
527
80
91
545
150
204
197
371
169
163
89
64
47
45
218
232
232
226
401
513
531
529
512
496
301
301
432
450
559
556
321
312
147
24
541
30
154
175
176
301
611
601
232
228
582
204
89
71
89
457
465
406
386
395
531
528
502
104
62
62
41
66
66
48
50
64
386
197
207
246
261
549
573
12
19
30
519
373
371
358
301
301
301
500
155
411
432
108
141
620
177
64
41
64
59
423
500
502
501
150
126
466
454
119
121
121
89
235
557
545
378
519
396
300
321
257
267
269
30
15
26
22
176
390
395
127
63
63
64
64
432
432
435
432
296
316
366
373
466
465
266
30
118
121
108
121
89
197
552
155
154
529
51
64
59
235
268
257
255
298
130
414
465
89
93
95
95
15
12
203
226
232
63
47
155
514
513
527
30
30
203
415
207
207
422
432
189
432
302
620
591
398
401
401
402
391
378
545
2

In [34]:
df2 = df.replace(list_clubs, rank_list)

In [35]:
df2 = df2.reset_index()
df2['index'] = df2['index'].apply(lambda x : x+1)
df2 = df2.rename(columns={'index': 'id'})

In [36]:
match_home_teams_list = []
for i in zip(df2['id'], df2['Equipe domicile'],df2['Score domicile']):
    match_home_teams_list.append(i)

In [37]:
for match_team in match_home_teams_list:
    cursor.execute("""INSERT INTO match_teams (match_id, team_id, home, goal) 
                VALUES (?,?,?,?)""", 
                (match_team[0], match_team[1], True, match_team[2]))
con.commit()

In [38]:
match_away_teams_list = []
for i in zip(df2['id'], df2['Equipe exterieure'],df2['Score extérieur']):
    match_away_teams_list.append(i)  

In [39]:
for match_team in match_away_teams_list:
    cursor.execute("""INSERT INTO match_teams (match_id, team_id, home, goal) 
                VALUES (?,?,?,?)""", 
                (match_team[0], match_team[1], False, match_team[2]))
con.commit()

In [40]:
cursor.execute("""SELECT * FROM match_teams""").fetchall()

[(1, 1, 12, 1, 0, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (2, 2, 20, 1, 0, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (3, 3, 1, 1, 1, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (4, 4, 3, 1, 2, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (5, 5, 16, 1, 3, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (6, 6, 19, 1, 4, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (7, 7, 9, 1, 2, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (8, 8, 8, 1, 2, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (9, 9, 2, 1, 1, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (10, 10, 5, 1, 0, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (11, 11, 4, 1, 4, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (12, 12, 6, 1, 2, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (13, 13, 15, 1, 0, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (14, 14, 14, 1, 0, None, '2021-07-15 06:18:56', '2021-07-15 06:18:56'),
 (15

In [45]:
df.to_pickle('Data/df.pkl')

In [46]:
df_players.to_pickle('Data/players.pkl')

In [47]:
df_teams.to_pickle('Data/df_teams.pkl')